In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from geo_dist_prep.data import GEONAMES_DB
from geo_dist_prep.schemas.geoname import GeoName

engine = create_engine(f"sqlite:///../{GEONAMES_DB}")

Session = sessionmaker(bind=engine)
session = Session()


In [ ]:
import pandas as pd

from sqlalchemy.sql import and_, select
from sqlalchemy import func, bindparam, Integer

southport = 26700924
lon, lat = -3.004175, 53.647599

def km_to_lon(lat, km):
    return km / (111.32 * func.cos(func.radians(lat) * func.pi() / 180))

def km_to_lat(km):
    return km / 111.32

def lat_to_km(lat):
    return lat * 111.32

def lon_to_km(lat, lon):
    return lon * 111.32 * func.cos(func.radians(lat))

def distance(place):
    """
    Returns the distance in km between two points on the Earth's surface.
    Place should be a table or subquery.columns with `lat` and `lon`.
    """
    return func.sqrt(
        func.pow(lat_to_km(GeoName.lat) - lat_to_km(place.lat), 2) + 
        func.pow(lon_to_km(GeoName.lat, GeoName.lon) - lon_to_km(place.lat, place.lon), 2)
    ).label("distance")

def nearby(lat, lon, km):
    """
    Returns a boolean expression that is true if a place is within `km` km of the
    given place. Place should be a table/subquery.columns with `lat` and `lon`.
    km should be a bindparam float.
    """
    in_box = and_(
        GeoName.lat >= lat - km_to_lat(km),
        GeoName.lat <= lat + km_to_lat(km),
        GeoName.lon >= lon - km_to_lon(lat, km),
        GeoName.lon <= lon + km_to_lon(lat, km)
    )
    #in_circle = distance(place) < km

    return in_box# and_(in_box, in_circle)

def grid_coord(place, grid_size: float):
    """
    Returns a unique integer for each grid square of size `grid_size` km.
    place should be a table/subquery.columns with `lat` and `lon`. grid_size
    should be a float.
    """
    y = func.floor((lat_to_km(place.lat) + 90 * 111.32) / grid_size)
    lon_km = lon_to_km(place.lat, grid_size)
    x = func.floor((lon_to_km(place.lat, place.lon) + 180 * 111.32) / lon_km)
    max_y = int(180 * 111.32 / grid_size) + 1
    
    return func.cast(y * max_y + x, Integer)

#place = session.query(GeoName).filter(GeoName.osm_id == bindparam('id')).subquery().c
query = session.query(GeoName)#.filter(nearby(53.647599, -3.004175, 35))
query = query.add_columns(grid_coord(GeoName, 2).label("grid_coord"))

q = query.statement.compile()
p = dict(q.params)
print(q)

df = pd.read_sql_query(q, engine, params=p)
df


In [ ]:
print(len(df))
smaller = df# df.sample(100)

print(len(smaller))

for i in smaller.to_dict(orient='records'):
    p['id'] = i['osm_id']
    a = pd.read_sql_query(q, engine, params=p)
    print(i['name'], len(a))
